In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# Define a CNN that will serve as one expert.
class ExpertCNN(nn.Module):
    def __init__(self, in_channels=16, num_classes=10):
        super(ExpertCNN, self).__init__()
        # Two convolutional layers with ReLU activations.
        self.conv1 = nn.Conv2d(in_channels, in_channels*2, kernel_size=3, padding=1)  # 1x28x28 -> 32x28x28
        self.conv2 = nn.Conv2d(in_channels*2, in_channels, kernel_size=3, padding=1) # 32x28x28 -> 64x28x28
        self.pool = nn.MaxPool2d(2, 2)                           # Downsample: 64x28x28 -> 64x14x14
        self.feature_dim = in_channels * 14 * 14
        # Final linear layer to output logits.
        self.fc = nn.Linear(self.feature_dim, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # Flatten the feature maps.
        x = self.fc(x)
        return x

In [3]:
# Define a CNN-MoE with multiple CNN experts.
class CNNMoE(nn.Module):
    def __init__(self, in_channels=1, num_experts=3, num_classes=10, top_k=1):
        super(CNNMoE, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=3, padding=1)
        self.num_experts = num_experts
        self.top_k = top_k
        # Create a ModuleList of CNN experts.
        self.experts = nn.ModuleList([ExpertCNN(in_channels=32, num_classes=num_classes)
                                      for _ in range(num_experts)])
        # A simple gating network that processes the input image and outputs weights.
        # Here we flatten the image and use a couple of linear layers.
        self.gate = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, num_experts)
        )

    def forward(self, x):
        # Compute gate logits and apply softmax to get probabilities.
        gate_logits = self.gate(x)           # Shape: [batch_size, num_experts]
        gate_probs = F.softmax(gate_logits, dim=1)
        
        # Top-k gating: select only the top k experts per sample.
        topk_values, topk_indices = torch.topk(gate_probs, self.top_k, dim=1)
        mask = torch.zeros_like(gate_probs)
        mask.scatter_(1, topk_indices, 1.0)
        sparse_gate = gate_probs * mask
        # Renormalize so that the selected experts sum to 1.
        sparse_gate = sparse_gate / (sparse_gate.sum(dim=1, keepdim=True) + 1e-8)
        sparse_gate = sparse_gate.unsqueeze(2)  # Shape: [batch_size, num_experts, 1]

        x = self.conv1(x)
        # Each expert produces a prediction.
        expert_outputs = [expert(x) for expert in self.experts]  # Each: [batch_size, num_classes]
        expert_outputs = torch.stack(expert_outputs, dim=1)        # Shape: [batch_size, num_experts, num_classes]

        # Weighted sum of experts' outputs.
        output = torch.sum(expert_outputs * sparse_gate, dim=1)     # Shape: [batch_size, num_classes]
        return output, sparse_gate


In [4]:
# Data Augmentation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [5]:
device = torch.device("cuda:6" if torch.cuda.is_available() else "cpu")
model = CNNMoE(num_experts=3, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [6]:
model.train()
for epoch in range(10):
    total_loss = 0.0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output, _ = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1, Loss: 0.1623
Epoch 2, Loss: 0.0471
Epoch 3, Loss: 0.0310
Epoch 4, Loss: 0.0258
Epoch 5, Loss: 0.0195
Epoch 6, Loss: 0.0144
Epoch 7, Loss: 0.0137
Epoch 8, Loss: 0.0112
Epoch 9, Loss: 0.0129
Epoch 10, Loss: 0.0144


In [7]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output, gate = model(data)
        test_loss += F.cross_entropy(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
print(f'\nTest set: Average loss: {test_loss:.4f}, '
        f'Accuracy: {correct}/{len(test_loader.dataset)} '
        f'({100. * correct / len(test_loader.dataset):.0f}%)\n')



Test set: Average loss: 0.0476, Accuracy: 9890/10000 (99%)

